In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

En este script, procedere a limpiar y procesar los datos. Estas tranformaciones forman parte del modelado oficial del proyecto.

Cada observacion dada en estas transformaciones se han visto analizadas previamente en los archivos del EDA.

- Estamos ante la segunda version del modelo de recomendacion.
- Los datos seran la fusion realizada previamente en el archivo 'processed_dataV1.0'

In [2]:
df_ = pd.read_parquet('../raw/data_model/dataset_process_trainV1.0.parquet')#Carga de datos

In [3]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15839932 entries, 0 to 15839931
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     float64
 1   movieId    int64  
 2   rating     float64
 3   timestamp  float64
 4   title      object 
 5   genres     object 
 6   tagId      int64  
 7   relevance  float64
 8   tag        object 
dtypes: float64(4), int64(2), object(3)
memory usage: 1.1+ GB


In [4]:
# df_, revisando nulos
df_.isna().sum()

userId       12
movieId       0
rating       12
timestamp    12
title         2
genres        2
tagId         0
relevance     0
tag           0
dtype: int64

In [8]:
# Los valores nulos de df_ los eliminaremos
def imputer_(df_):
    df_ratings_copy = df_.copy()
    df_ratings_copy=df_ratings_copy.dropna()

    return df_ratings_copy

In [9]:
df_=imputer_(df_)# Eliminamos los valores nulos

In [10]:
df_.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
tagId        0
relevance    0
tag          0
dtype: int64

In [11]:
# df_ratings, revisando duplicados en titulos (no queremos el mismo titulo mas de 1 vez)
df_.nunique() # Comprobamos los valores duplicados por columnas

userId         138493
movieId         10369
rating             10
timestamp    12537611
title           10369
genres            930
tagId             853
relevance        1018
tag               853
dtype: int64

El dataset se encuentran correctamente con respecto a los duplicados.
- En title no tenemos ningun titulo igual repetido.
- En tags tenemos la misma cantidad de tag como de su id.

In [12]:
# Transformamos las variables del dataset 
# Timestamp, convertimos a datetime (referente a fecha de valoracion)
def timestamp_to_datetime_df_(df_):
    df_ratings_copy = df_.copy()
    # Convertimos el timestamp a datetime, luego aplicamos el formato año/mes/dia devolviendo un string
    df_ratings_copy['date_rating'] = df_ratings_copy['timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
    # Transformamos a Date el String
    df_ratings_copy['date_rating'] = df_ratings_copy['date_rating'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # Eliminamos variable timestamp
    df_ratings_copy.drop('timestamp', axis=1, inplace=True)

    return df_ratings_copy

In [13]:
df_=timestamp_to_datetime_df_(df_) # Transforma timestamp a datetime (las fechas no valoradas se veran asi:"1970-01-01")

In [14]:
# Transformamos las variables del dataset df_
# date_rating, separamos año,mes y dia en variables independientes
def date_df_indepenDates(df_):
    df_ratings_copy = df_.copy()
    # Extraemos en variables sepadas el año, mes y dia de date_rating
    df_ratings_copy['year_rate'] = df_ratings_copy['date_rating'].dt.year
    df_ratings_copy['month_rate'] = df_ratings_copy['date_rating'].dt.month
    df_ratings_copy['day_rate'] = df_ratings_copy['date_rating'].dt.day
    # Eliminamos variable timestamp
    df_ratings_copy.drop('date_rating', axis=1, inplace=True)
    return df_ratings_copy

In [15]:
df_=date_df_indepenDates(df_)# Extraemos fechas independientes

In [23]:
# Aleatoriamente modificamos 5 peliculas por usuario para simular que no la han visto.
def poner_no_vistas(grupo):
    # Número de películas a poner como no vistas
    num_no_vistas = 5
    
    # Selecciona aleatoriamente algunas películas para poner como no vistas
    peliculas_no_vistas = np.random.choice(grupo['movieId'].unique(), size=num_no_vistas, replace=False)

    # Actualiza el grupo
    mask = grupo['movieId'].isin(peliculas_no_vistas)
    grupo.loc[mask, ['rating', 'year_rate', 'month_rate' ,'day_rate']] = 0

    return grupo

In [24]:
df_=df_.groupby('userId').apply(poner_no_vistas)# Llamada a peliculas no vistas

In [28]:
# Resetea el índice del DataFrame
def reset_grupo_index(df_):
    df_ = df_.reset_index(drop=True)
    return df_

In [29]:
df_=reset_grupo_index(df_)# llamada a reset index

In [40]:
# Transformamos las variables del dataset df_
# title, Extraemos el año de publicacion de cada pelicula en una variable nueva
def launchYear_title_df_(df_):
    df_ratings_copy = df_.copy()
    # Extraemos con expresiones regulares los años del title, rellenamos los nulos con 0 y casteamos a int
    df_ratings_copy['launch_year'] = df_ratings_copy['title'].str.extract(r'\((\d{4})\)', expand=False).fillna('0').astype(int)
    # Eliminamos el año y cualquier contenido entre paréntesis en la columna 'title'
    df_ratings_copy['title'] = df_ratings_copy['title'].apply(lambda x: x.split(' (')[0])
    return df_ratings_copy

In [41]:
df_=launchYear_title_df_(df_) # Extrae año del titulo

In [42]:
# Transformamos las variables del dataset df_
# Convertimos userId a entero
def int_userId_df_(df_):
    df_ratings_copy=df_.copy()
    # Convertir 'userId' a int
    df_ratings_copy['userId'] = pd.to_numeric(df_ratings_copy['userId'], errors='coerce').astype('int64')
    return df_ratings_copy

In [43]:
df_=int_userId_df_(df_)# Convierte userId a entero

In [44]:
# Transformamos las variables del dataset df_
# Extraemos la longitud de cada title y lo añadimos como nueva variable
def lenTitle_df_(df_):
    df_ratings_copy=df_.copy()
    #Extraemos la longitud de cada uno de los titulos
    df_ratings_copy['len_title']=df_ratings_copy['title'].apply(lambda x: len(x))

    return df_ratings_copy

In [45]:
df_=lenTitle_df_(df_) # Extraemos longitud del titulo

In [46]:
# Transformamos las variables del dataset df_
# Extraemos  el genero principal de cada movieId (se considera principal al primero que aparece en genres antes del '|')
def mainGenre_df_(df_):
    df_ratings_copy=df_.copy()
    # Extraemos el primer genero de la columna genres y lo almacenamos en una nueva variable
    df_ratings_copy['main_genre']=df_ratings_copy['genres'].apply(lambda x: x.split('|')[0])
    return df_ratings_copy

In [47]:
df_=mainGenre_df_(df_) #Extraemos genero principal

In [48]:
# Transformamos las variables del dataset df_
# Extraemos el genero secundario de cada movieId (se considera secundario al segundo que aparece en genres antes del '|')
def secundaryGenre_df_(df_):
    df_ratings_copy=df_.copy()
    df_ratings_copy['secondary_genre'] = df_ratings_copy['genres'].apply(lambda x: x.split('|')[1] if len(x.split('|')) >1 else 'no_secundary')
    return df_ratings_copy

In [49]:
df_ = secundaryGenre_df_(df_)#Extraemos genero secundario

In [50]:
# Transformamos las variables del dataset df_
# Extraemos el genero tercero de cada movieId (se considera secundario al tercero que aparece en genres antes del '|')
def thirdGenre_df_(df_):
    df_ratings_copy=df_.copy()
    df_ratings_copy['third_genre'] = df_ratings_copy['genres'].apply(lambda x: x.split('|')[2] if len(x.split('|')) >2 else 'no_third')
    return df_ratings_copy

In [51]:
df_=thirdGenre_df_(df_)#Extraemos genero terciario

In [52]:
def valoration_df(df_):
    df_ratings_copy=df_.copy()
    # Gestionamos el rating en funcion de sus valores y añadimos cuatro categorias: 'excellent','good','bad','unknown'
    df_ratings_copy['valoration']=df_ratings_copy['rating'].apply(lambda x: 'excellent' if x == 5 else('good' if x >= 3 else ('unknown' if x == 0 else 'bad')))
    return df_ratings_copy

In [53]:
df_=valoration_df(df_)# Categorias en funcion de rating

In [54]:
def drop_columns(df_):
    df_ratings_copy=df_.copy()
    df_ratings_copy.drop(['title','genres','tag'], axis=1, inplace=True)
    return df_ratings_copy

In [55]:
df_= drop_columns(df_)

In [56]:
df_

,userId,movieId,rating,tagId,relevance,year_rate,month_rate,day_rate,launch_year,len_title,main_genre,secondary_genre,third_genre,valoration
0,1,2,3.5,29,0.409360,2005,4,3,1995,7,Adventure,Children,Fantasy,good
1,1,29,3.5,287,1.159197,2005,4,3,1995,26,Adventure,Drama,Fantasy,good
2,1,32,3.5,1027,1.073053,2005,4,3,1995,14,Mystery,Sci-Fi,Thriller,good
3,1,47,3.5,300,0.579532,2005,4,3,1995,5,Mystery,Thriller,no_third,good
4,1,50,3.5,758,1.196798,2005,4,3,1995,19,Crime,Mystery,Thriller,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15839915,138493,68319,4.5,987,1.048844,2009,12,7,2009,24,Action,Sci-Fi,Thriller,good
15839916,138493,69526,4.5,437,1.184218,2009,12,3,2009,35,Action,Adventure,Sci-Fi,good
15839917,138493,69644,3.0,64,0.569222,2009,12,7,2009,30,Action,Adventure,Animation,good
15839918,138493,70286,5.0,45,0.884267,2009,11,13,2009,10,Mystery,Sci-Fi,Thriller,excellent


Dataset procesado version 1.5